<a href="https://colab.research.google.com/github/omerburst/Basin-data-GEEmap/blob/main/Basin_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Initializing
```
# This is formatted as code
```



In [1]:
#Importing packages

import ee
import pandas as pd
import math

In [2]:
#Login through your personal gee account
import ee

# Trigger the authentication flow.
ee.Authenticate()

# change to the name of your gee project
ee.Initialize(project='your project name')

In [3]:
#Installing the package of GEE
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00


In [4]:
#Using MODIS projection to create uniform scale for all layers
modis_projection = 'EPSG:4326'
modis_scale = 250

# Choosing your basin of interest

In [6]:
import geemap
import folium
import ee


# Create a map instance
map = geemap.Map()

# Add a basemap
map.add_basemap('SATELLITE')

# Create the FeatureViewLayer and set the visualization parameters
asset_id = 'WWF/HydroSHEDS/v1/Basins/hybas_8'
fvLayer = ee.FeatureCollection(asset_id)
visParams = {'color': 'FFFACD', 'lineWidth': 1}

map.addLayer(fvLayer, visParams, 'Basins')

#adding control of layer on the upper right side of map
map.addLayerControl()

map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
roi = map.draw_last_feature

latlon = roi.getInfo()

long = latlon['geometry']["coordinates"][0]
lat = latlon['geometry']["coordinates"][1]

# Convert the latitude and longitude to a Point geometry
point = {'type':'Point', 'coordinates':[ long, lat]};

# Create an Earth Engine geometry object from the GeoJSON
point_geometry = ee.Geometry(point)



#Creating map with relevant layers of your basin

In [8]:
# Palette for CTI layer
cti_palette = ['white', 'blue', 'green', 'yellow', 'red']

# Palette for SPI layer
spi_palette = ['purple', 'blue', 'aqua', 'yellow', 'orange', 'red']

# Palette for STI layer
sti_palette = ['navy', 'cyan', 'lime', 'yellow', 'red']

In [9]:


# Create a map instance
map1 = geemap.Map()

# Add a basemap
map1.add_basemap('SATELLITE')

## Adding basins layer

scale = modis_scale



## Adding dem layer

dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select(["elevation"])

##Adding slope and aspect

slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

slope = slope.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

##Adding TPI

dataset = ee.Image('CSP/ERGo/1_0/Global/SRTM_mTPI');
srtmMtpi = dataset.select('elevation');
srtmMtpiVis = {
  'min': -100.0,
  'max': 100.0,
  'palette' : ['0b1eff', '4be450', 'fffca4', 'ffa011', 'ff0000']
}

##Adding CN

GCN250_Average =  ee.Image("users/jaafarhadi/GCN250/GCN250Average").select('b1').rename('average');
GCN250_Dry =  ee.Image("users/jaafarhadi/GCN250/GCN250Dry").select('b1').rename('dry');
GCN250_Wet = ee.Image("users/jaafarhadi/GCN250/GCN250Wet").select('b1').rename('wet');

visCN = {
  'min':40,
  'max':75,
  'palette': ['Red','SandyBrown','Yellow','LimeGreen', 'Blue','DarkBlue']

}


##Adding HAND

hand_30 = ee.Image("users/gena/GlobalHAND/30m/hand-1000")

paletteHand = ['023858', '006837', '1a9850', '66bd63', 'a6d96a', 'd9ef8b', 'ffffbf', 'fee08b', 'fdae61', 'f46d43', 'd73027'];
visHAND = {'min': 1, 'max': 150, 'palette': paletteHand}

## Adding TWI

dataset = ee.Image('WWF/HydroSHEDS/15ACC');
flowAccumulation = dataset.select('b1');

scaleFactor = 463.83 * 463.83 / 1000000

contributingArea = flowAccumulation.multiply(scaleFactor)

#contributingArea = resample(contributingArea).reproject(crs=modis_projection, scale=scale)
contributingArea = contributingArea.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

beta = slope.divide(180).multiply(math.pi) #convert to radians

twi = (contributingArea.divide(beta.tan())).log()

visTWI = {
  'min':0,
  'max':20,
  'palette': ['Red','SandyBrown','Yellow','LimeGreen', 'Blue','DarkBlue']

}


##Adding CTI

cti = ee.ImageCollection('projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/cti')
original_projection = ee.Image(cti.first()).projection()

cti_palette = ['8B4513', 'A0522D', 'CD853F', 'D2691E', 'FFA500', '00BFFF', '1E90FF', '4169E1', '000080']

spi = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/spi")
spi_projection = ee.Image(spi.first()).projection()

sti = ee.ImageCollection("projects/sat-io/open-datasets/HYDROGRAPHY90/flow_index/sti")
sti_projection = ee.Image(sti.first()).projection()

# Filter the FeatureCollection using the point geometry
chosen_basin = fvLayer.filterBounds(point_geometry)

# Define the dams collection
dams = ee.FeatureCollection("projects/sat-io/open-datasets/GOODD/GOOD2_dams")

# Filter the FeatureCollection using the point geometry
chosen_dams = dams.filterBounds(chosen_basin.geometry())

# Clip the slope layer to the extent of basins_with_stations
slope_clipped = slope.clip(chosen_basin.geometry())

#slope_clipped = slope_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

tpi_clipped = srtmMtpi.clip(chosen_basin.geometry())

tpi_clipped = tpi_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

GCN250_Dry_clipped = GCN250_Dry.clip(chosen_basin.geometry())

GCN250_Dry_clipped = GCN250_Dry_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

hand_30_clipped = hand_30.clip(chosen_basin.geometry())

hand_30_clipped = hand_30_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

twi_clipped = twi.clip(chosen_basin.geometry())

cti = cti.mosaic().reproject(crs=original_projection).divide(1e8)
cti_clipped = cti.clip(chosen_basin.geometry())
cti_clipped = cti_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)
# cti_clippe = cti_clipped.setDefaultProjection(crs=modis_projection, scale=250)

spi = spi.mosaic().reproject(crs=spi_projection).divide(1e3)
spi_clipped = spi.clip(chosen_basin.geometry())
spi_clipped = spi_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

sti = sti.mosaic().reproject(crs=sti_projection).divide(1e3)
sti_clipped = sti.clip(chosen_basin.geometry())
sti_clipped = sti_clipped.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=modis_projection, scale=scale)

Land_cover = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select('discrete_classification')

Land_cover = Land_cover.reduceResolution(reducer=ee.Reducer.mode(), bestEffort=True).reproject(crs=modis_projection, scale=scale)
Land_cover_clipped = Land_cover.clip(chosen_basin.geometry())

## Adding globgm-heads-lower-layer-ss layer

globgm_heads_lower_layer_ss = ee.Image('projects/sat-io/open-datasets/GLOBGM/STEADY-STATE/globgm-heads-lower-layer-ss')

globgm_vis_params = {
    'min': -2,
    'max': 1200,
    'palette': ["001219", "005f73", "0a9396", "94d2bd", "e9d8a6", "ee9b00", "ca6702", "bb3e03", "ae2012", "9b2226"]
}

# Clip the globgm-heads-lower-layer-ss layer to the extent of chosen_basin
globgm_heads_clipped = globgm_heads_lower_layer_ss.clip(chosen_basin.geometry())

# Adding hack order
order_hack = ee.ImageCollection('projects/sat-io/open-datasets/HYDROGRAPHY90/stream-order/order_hack')
order_hack_projection = ee.Image(order_hack.first()).projection()

order_hack = order_hack.mosaic().reproject(crs=order_hack_projection)

# Clip the order_hack layer to the extent of chosen_basin
order_hack_clipped = order_hack.clip(chosen_basin.geometry())

# Adding layers to map according to sequence
map1.addLayer(slope_clipped, {min: 0, max: 30}, 'slope')

# Add the globgm-heads-lower-layer-ss layer to the map
map1.addLayer(globgm_heads_clipped, globgm_vis_params, 'Computed water table depth [m] (lower layer only) 2015')

map1.addLayer(Land_cover_clipped, {}, 'Land Cover')

map1.addLayer(tpi_clipped,srtmMtpiVis, 'TPI')

map1.addLayer(GCN250_Dry_clipped, visCN, 'CN Dry')

map1.addLayer(hand_30_clipped, visHAND, 'HAND')

map1.addLayer(twi_clipped, visTWI, 'TWI')

map1.addLayer(cti_clipped,{'min' : -3.554,'max':3.161,'palette': cti_palette}, 'CTI')
map1.addLayer(spi_clipped,{'min':-78.418,'max':83.271,'palette':spi_palette}, 'SPI')
map1.addLayer(sti_clipped,{'min':-0.552,'max':0.792,'palette': sti_palette},'STI')

map1.addLayer(order_hack_clipped, {'min': 1, 'max': 4, 'palette': ['black', 'darkblue', 'blue', 'turquoise']}, 'river_order_layer')  # Add river order layer

# Add the dams layer to the map with a red color palette and label "GOODD Dams"
map1.addLayer(chosen_dams, {"color": "FF0000"}, 'GOODD Dams')

map1.add_legend(builtin_legend="COPERNICUS/Landcover/100m/Proba-V/Global", title="Land Cover")

legend_keys = ["One", "Two", "Three", "Four"]
legend_colors = ["#000000", "#00008B", "#0000FF", "#40E0D0"]

map1.add_legend(title = 'Order Hack', keys=legend_keys, colors=legend_colors, position="bottomleft")

map1.addLayerControl()   # Adding control of layer on the upper right side of map

# Set the map center to the coordinates of point_geometry with an appropriate zoom level
map1.setCenter(long, lat, zoom=10)

# Display the map
map1


Map(center=[43.171117, 11.899762], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#Downloading the layers as GeoTIFF

In [13]:
layer_dict = {slope_clipped : 'slope_clipped', Land_cover_clipped : 'Land_cover_clipped',
              tpi_clipped : 'tpi_clipped', GCN250_Dry_clipped : 'GCN250_Dry_clipped',
              hand_30_clipped : 'hand_30_clipped' ,twi_clipped : 'twi_clipped',
              cti_clipped : 'cti_clipped', sti_clipped : 'sti_clipped', spi_clipped : 'spi_clipped',
               order_hack_clipped : 'order_hack_clipped', globgm_heads_clipped : 'globgm_heads_clipped' }

#Insret the name of your basin
basin_name = "italy"

#Change the path to your google drive folder
for layer , name in layer_dict.items():
  geemap.ee_export_image_to_drive(
      layer, description= name + '_' + basin_name, folder='/content/drive/MyDrive/yourfolder', region= chosen_basin.first().geometry(), scale=scale, fileFormat='GeoTIFF',
                                  crs='EPSG:4326' , maxPixels=1e13
  )

  # Export vector layer (chosen_dams)
geemap.ee_export_vector_to_drive(
    chosen_dams, description='chosen_dams_' + basin_name, folder='/content/drive/MyDrive/yourfolder',
    fileFormat='GeoJSON'
)